In [1]:
import os
import pandas as pd
import numpy as np
import torch as t
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
import scipy.io

In [2]:
data_dir = '/Users/tyler/Datasets/deepsat-sat6/'

In [3]:
os.listdir(data_dir)

['y_test_sat6.csv',
 'sat-6-full.mat',
 'X_train_sat6.csv',
 'sat6annotations.csv',
 'y_train_sat6.csv',
 'X_test_sat6.csv']

In [11]:
class SatelliteDataset(Dataset):
    def __init__(self, X_csv, y_csv, transform=None):
        self.instances = pd.read_csv(X_csv)
        self.labels = pd.read_csv(y_csv)
        self.transform = transform
    
    def __len__(self): return len(self.labels)
    
    def __getitem__(self, idx):
        img = self.instances.iloc[idx].values.reshape(-1, 28, 28, 4).clip(0, 255).astype(np.uint8).squeeze(axis=0)
        label = self.labels.iloc[idx].values
        return {'chip': img, 'label': label}

In [12]:
training_set = SatelliteDataset(X_csv=data_dir+'X_train_sat6.csv', y_csv=data_dir+'y_train_sat6.csv')

In [13]:
validation_set = SatelliteDataset(X_csv=data_dir+'X_test_sat6.csv', y_csv=data_dir+'y_test_sat6.csv')

In [19]:
sample = next(iter(training_set))
sample

{'chip': array([[[177, 152, 129, 183],
         [180, 154, 128, 182],
         [181, 156, 129, 180],
         ...,
         [184, 164, 133, 177],
         [185, 164, 140, 179],
         [192, 172, 135, 179]],
 
        [[176, 152, 127, 184],
         [181, 155, 132, 183],
         [178, 152, 131, 182],
         ...,
         [181, 165, 137, 181],
         [185, 167, 141, 180],
         [194, 173, 147, 180]],
 
        [[178, 156, 130, 186],
         [185, 157, 131, 184],
         [185, 158, 131, 183],
         ...,
         [184, 162, 136, 181],
         [190, 170, 145, 182],
         [196, 177, 145, 181]],
 
        ...,
 
        [[184, 168, 143, 178],
         [192, 170, 145, 176],
         [196, 171, 143, 174],
         ...,
         [183, 166, 139, 178],
         [180, 165, 134, 178],
         [176, 158, 134, 174]],
 
        [[186, 164, 137, 174],
         [188, 166, 135, 173],
         [190, 169, 139, 174],
         ...,
         [178, 161, 135, 173],
         [182, 161, 136, 17

In [22]:
tfms = tv.transforms.Compose([
    tv.transforms.RandomHorizontalFlip(),
    tv.transforms.RandomVerticalFlip(),
    tv.transforms.RandomRotation(90),
    tv.transforms.ToTensor()
])

In [23]:
train_loader = DataLoader(
    training_set,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

In [40]:
class SmallCNN(t.nn.Module):
    def __init__(self):
        super(SmallCNN, self).__init__()
        self.conv1 = t.nn.Sequential(
            t.nn.Conv2d(
                in_channels=28,
                out_channels=32,
                kernel_size=3
            ),
            t.nn.BatchNorm2d(32),
            t.nn.ReLU(),
            t.nn.Conv2d(
                in_channels=32,
                out_channels=32,
                kernel_size=3
            ),
            t.nn.BatchNorm2d(32),
            t.nn.ReLU(),
        )
        self.conv2 = t.nn.Sequential(
            t.nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3
            ),
            t.nn.BatchNorm2d(64),
            t.nn.ReLU(),
            t.nn.Conv2d(
                in_channels=64,
                out_channels=64,
                kernel_size=3
            ),
            t.nn.BatchNorm2d(64),
            t.nn.ReLU(),
        )
    
    def forward(self, x):
        x = self.conv1(x) # 
        x = t.nn.MaxPool2d(2)
        x = self.conv2(x)
        x = t.nn.MaxPool2d(2)
        x = x.view(x.size(0), -1) # flatten
        print(x.size())
        x = t.nn.Linear(3136, 512) # need to calculate fc input size. I think it's 64 * 7 * 7 = 3136
        x = t.nn.BatchNorm1d(512)
        x = t.nn.ReLU()
        x = t.nn.Dropout2d(0.2)
        x = t.nn.Linear(512, 6)
        output = t.nn.Softmax(x)

In [41]:
model = SmallCNN()
optim = t.optim.SGD(lr=0.001, momentum=0.9, params=model.parameters())
crit = t.nn.CrossEntropyLoss()

In [45]:
EPOCHS = 1
for e in range(EPOCHS):
    for s, inst in enumerate(train_loader):
        chip = inst['chip'].float()
        output = model(chip)
        loss = crit(output, inst['label'])
        optim.zero_grad()
        loss.backward()
        optim.step()

RuntimeError: std::exception